In [44]:
!pip install llama-index
!pip install docx2txt
!pip install pypdf
!pip install google-generativeai
!pip install transformers
!pip install llama_index.embeddings.huggingface
%pip install llama-index-llms-google-genai llama-index

## Load Data

In [45]:
mkdir data

mkdir: cannot create directory ‘data’: File exists


In [46]:
from llama_index.core import SimpleDirectoryReader

In [47]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader

documents = SimpleDirectoryReader("data").load_data()

In [48]:
documents

[Document(id_='25347038-6b11-4f84-9bf1-3b52cf15289f', embedding=None, metadata={'page_label': '1', 'file_name': 'BNS.pdf', 'file_path': '/content/data/BNS.pdf', 'file_type': 'application/pdf', 'file_size': 871650, 'creation_date': '2025-08-20', 'last_modified_date': '2025-08-20'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text_resource=MediaResource(embeddings=None, data=None, text='1 \n \nTHE BHARATIYA NYAYA SANHITA, 2023 \n________ \nARRANGEMENT OF SECTIONS \n________ \nCHAPTER I \nPRELIMINARY \nSECTIONS \n1. Short title, commencement and application. \n2. Definitions. \n3. General explanations. \nCHAPTER II \nOF PUNISHMENTS \n4. Punishments. \n5. Commutation of sentence. \n6. Fractions of te

## Split text into Small Chunks

In [ ]:
import os

os.environ["GOOGLE_API_KEY"] = ""
os.environ["HF_TOKEN"] = ""

In [50]:
# Calling LLM
from llama_index.llms.google_genai import GoogleGenAI
from llama_index.core import Settings

llm = GoogleGenAI(
    model="gemini-2.5-flash")
Settings.llm = llm

 a vector is an array of numbers that represents complex data like text, images, or audio as numerical features. AI algorithms use these numerical representations in vector databases to find similar data, understand context, and power applications such as semantic search, recommendations, and large language models. This process, known as vector embedding, converts unstructured data into a mathematical format that machine learning models can process and analyze. 

In [51]:
# Embedding model

from llama_index.embeddings.huggingface import HuggingFaceEmbedding
embedding_model = HuggingFaceEmbedding(model_name="BAAI/bge-base-en")

In [52]:
# Splitter
from llama_index.core.node_parser import SentenceSplitter

splitter = SentenceSplitter(chunk_size=1000, chunk_overlap=150)

In [53]:
from llama_index.core import VectorStoreIndex

index = VectorStoreIndex.from_documents(documents, transformations=[splitter], llm=llm, embed_model=embedding_model)

Storing and Loading it in local

In [54]:
# Storing the index

index.storage_context.persist()

In [55]:
# Loading the index
#storage_context = StorageContext.from_defaults(persist_dir="./storage")
#index = load_index_from_storage(storage_context)

## QNA

In [56]:
prompt = """
"You are a legal assistant specialized in the Indian Constitution. "
"Always answer strictly based on the provided context. "
"If the answer is not in the context, say 'I cannot find this information in the Constitution.'
"Never provide personal opinions or hallucinate."
"""

In [57]:
from llama_index.core.prompts import PromptTemplate

qa_prompt = PromptTemplate(prompt)

In [58]:
chat_engine = index.as_chat_engine(text_qa_prompt = qa_prompt)

In [67]:
response = chat_engine.chat("culpable homicide")
print(response)

The documents provide detailed information about "culpable homicide," particularly distinguishing it from murder and outlining its punishments.

Here's a breakdown:

**1. Definition and Relation to Murder (Section 102, page 45):**
Culpable homicide is committed when a person causes the death of another, even if they did not intend or know themselves to be likely to cause the death of *that specific person*. In such a case, the culpable homicide is treated as if the offender had caused the death of the person whose death they *did* intend or know themselves to be likely to cause. This means the intent or knowledge regarding one person can be transferred to the actual victim.

**2. Punishment for Culpable Homicide Not Amounting to Murder (Section 105, page 45):**
The punishment for culpable homicide that does not qualify as murder depends on the offender's intent or knowledge:

*   **If the act is done with the intention of causing death, or of causing such bodily injury as is likely to 